# TESTE: Aplicar Hierarquias de Separação

**Carregar resultado do analise_combinacoes até ter df_limpo pronto.**

In [ ]:
# Assumindo que já corri analise_combinacoes.ipynb até célula 7
# e tenho df_limpo disponível

import pandas as pd
import numpy as np

print(f'df_limpo shape: {df_limpo.shape}')
print(f'Colaboradores: {df_limpo["login_colaborador"].nunique()}')

In [ ]:
# VER combinações que existem ANTES de eliminar
print('=== COMBINAÇÕES EXISTENTES ===')

dias_mult = df_limpo.groupby(['login_colaborador', 'Data']).size()
dias_mult = dias_mult[dias_mult > 1].reset_index()

print(f'Dias com múltiplos registos: {len(dias_mult):,}')

# Ver combinações REAIS
test_combos = []
for i in range(min(10, len(dias_mult))):
    login = dias_mult.iloc[i]['login_colaborador']
    data = dias_mult.iloc[i]['Data']
    
    regs = df_limpo[(df_limpo['login_colaborador']==login) & (df_limpo['Data']==data)]
    n1 = ' + '.join(sorted(regs['Nivel 1'].unique()))
    
    test_combos.append(n1)
    print(f'{i+1}. {n1}')

print(f'\nTESTE: Quantos têm "Atraso + Trabalho Pago"?')
print(test_combos.count('Atraso + Trabalho Pago'))

In [ ]:
# VERSÃO CORRECTA: usar merge para identificar rápido
print('\n=== MÉTODO RÁPIDO ===')

# Criar tabela de combinações
dias_mult_idx = list(zip(dias_mult['login_colaborador'], dias_mult['Data']))
df_mult_only = df_limpo[df_limpo.set_index(['login_colaborador', 'Data']).index.isin(dias_mult_idx)]

print(f'Registos de dias múltiplos: {len(df_mult_only):,}')

# Agrupar
df_combos = df_mult_only.groupby(['login_colaborador', 'Data']).agg({
    'Nivel 1': lambda x: ' + '.join(sorted(x.unique())),
    'Nivel 2': lambda x: ' + '.join(sorted(x.unique()))
}).reset_index()

df_combos.columns = ['login_colaborador', 'Data', 'combo_n1', 'combo_n2']

print(f'\nCombinações Nivel 1:')
print(df_combos['combo_n1'].value_counts())

In [ ]:
# AGORA SIM: Identificar casos a eliminar
print('\n=== CASOS A ELIMINAR ===')

# Nivel 1
eliminar_n1 = df_combos[
    df_combos['combo_n1'].isin([
        'Falta Injustificada + Trabalho Pago',
        'Falta Justificada + Trabalho Pago',
        'Atraso + Falta Justificada'
    ])
]

print(f'Nivel 1 - casos raros: {len(eliminar_n1)}')
if len(eliminar_n1) > 0:
    print(eliminar_n1['combo_n1'].value_counts())

# Nivel 2
patterns = [
    'Feriado \\+ Presença',
    'Férias \\+ Presença',
    'Folga \\+ Presença',
    'Falta Injustificada \\+ Presença',
    'Assistência Familiar \\+ Presença',
    'Atraso \\+ Ausência Médica',
    'Ausência Justificada \\+ Presença'
]

eliminar_n2 = df_combos[
    df_combos['combo_n2'].str.contains('|'.join(patterns), na=False, regex=True)
]

print(f'\nNivel 2 - problemas: {len(eliminar_n2)}')
if len(eliminar_n2) > 0:
    print(eliminar_n2['combo_n2'].value_counts().head(10))

# TOTAL
eliminar_total = pd.concat([eliminar_n1[['login_colaborador', 'Data']], 
                             eliminar_n2[['login_colaborador', 'Data']]]).drop_duplicates()

print(f'\n🔴 TOTAL A ELIMINAR: {len(eliminar_total)} dias')

In [ ]:
# ELIMINAR
print('\n=== ELIMINANDO ===')

idx_eliminar = df_limpo.set_index(['login_colaborador', 'Data']).index.isin(
    list(zip(eliminar_total['login_colaborador'], eliminar_total['Data']))
)

print(f'Registos a eliminar: {idx_eliminar.sum()}')

df_limpo_v2 = df_limpo[~idx_eliminar].copy()

print(f'✓ df_limpo: {len(df_limpo):,} → {len(df_limpo_v2):,}')

In [ ]:
# SEPARAR ATRASOS
print('\n=== SEPARANDO ATRASOS ===')

dias_atraso = df_combos[df_combos['combo_n1'] == 'Atraso + Trabalho Pago']

print(f'Dias "Atraso + Trabalho Pago": {len(dias_atraso):,}')

# df_atrasos: só linhas onde Nivel 1 = 'Atraso'
idx_atrasos = df_limpo_v2.set_index(['login_colaborador', 'Data']).index.isin(
    list(zip(dias_atraso['login_colaborador'], dias_atraso['Data']))
) & (df_limpo_v2['Nivel 1'] == 'Atraso')

df_atrasos = df_limpo_v2[idx_atrasos].copy()

print(f'✓ df_atrasos: {len(df_atrasos):,} registos')

# Remover do principal
df_principal = df_limpo_v2[~idx_atrasos].copy()

print(f'✓ df_principal: {len(df_principal):,} registos (sem atrasos)')

In [ ]:
# AGREGAR
print('\n=== AGREGANDO ===')

agg_rules = {
    'nome_colaborador': 'first',
    'categoria_profissional': 'first',
    'segmento_processado_codigo': lambda x: ', '.join(sorted(x.unique())),
    'Nivel 1': lambda x: ', '.join(sorted(x.unique())),
    'Nivel 2': lambda x: ', '.join(sorted(x.unique())),
}

# Adicionar campos opcionais
for col in ['operacao', 'Activo?', 'DtActivacao', 'DtDesactivacao']:
    if col in df_principal.columns:
        agg_rules[col] = 'first'

# Agregar principal
df = df_principal.groupby(['login_colaborador', 'Data']).agg(agg_rules).reset_index()
print(f'✓ df (principal): {len(df):,} dias-colaborador')

# Agregar atrasos
df_atrasos_agg = df_atrasos.groupby(['login_colaborador', 'Data']).agg(agg_rules).reset_index()
print(f'✓ df_atrasos: {len(df_atrasos_agg):,} dias-colaborador')

In [ ]:
# VERIFICAR
print('\n' + '='*70)
print('RESUMO FINAL')
print('='*70)
print(f'df (principal)      : {len(df):,} dias')
print(f'df_atrasos          : {len(df_atrasos_agg):,} dias')
print(f'Dias eliminados     : {len(eliminar_total)}')
print('='*70)

# Ver distribuição Nivel 1 no principal
print('\nDistribuição Nivel 1 (df principal):')
print(df['Nivel 1'].value_counts().head(10))